# 4 Predictive Analytic tasks: Anomaly
Perform anomaly detection for requests per department per day in the first 6 months


In [1]:
import sqldf
import pandas as pd
import helper.functions as hf
from datetime import datetime
import plotly.express as px

In [3]:
# load data
col_types = {
    'notification_number':str,
    'reference_number':str
    }
date_cols = ['creation_timestamp','completion_timestamp']
df = pd.read_csv('data/raw/sr_hex.csv', parse_dates=date_cols,dtype=col_types)
df['date'] = pd.to_datetime(df['creation_timestamp'].dt.date, format='%Y-%m-%d')

In [4]:
query = '''
select department,
       date,
       count(distinct notification_number) req
from df
group by department, date
having date < '2020-07-01' and department is not null
order by department, date
'''

df_test = sqldf.run(query)

In [5]:
fig = px.box(df_test, x="department", y="req", log_y=False, width=800, height=800)
fig.show()

# Interpret
The top for departments in terms of service requests is Electricity, Distribution, commercial and solid waste.


In [6]:
from sklearn.ensemble import IsolationForest as IF

res = []

# We'll choose a 5% contamination, but this is a parameter we could play with
g = df_test.groupby(['department'])
for name, data in g:
    if data.shape[0] > 5:
        x = data[['req']]
        clf = IF(n_estimators=100, max_samples='auto', contamination=float(.05), max_features=1.0, bootstrap=False, n_jobs=-1)
        clf.fit(x.values)
        x_dates = data[['date']]
        pred = pd.DataFrame({'department':name,
                            'date':x_dates['date'],
                            'anomaly':clf.predict(x.values),
                            'scores':clf.decision_function(x.values),
                            'req':x['req']})
        pred['request_cnt'] = x['req'].sum()
        res.append(pred)


In [7]:
res = pd.concat(res)

# Let's see who had the most anomalies by department


In [8]:
query = '''
select department,
       case when anomaly = -1 then 1.0 else 0.0 end anom_cnt
from res
'''

sum_df = sqldf.run(query)

query = '''
select department,
       sum(anom_cnt) anom_cnt,
       count(*) cnt,
       sum(anom_cnt)/count(*)*100 anom_per
from sum_df
group by department
'''

sum_df = sqldf.run(query)
sum_df.sort_values('anom_per')

,department,anom_cnt,cnt,anom_per
11,Social Development & Early Childhood Development,6.0,139,4.316547
14,Transport Planning & Network Management,8.0,173,4.624277
15,Valuations,5.0,103,4.854369
8,Recreation and Parks,9.0,180,5.000000
2,Customer Relations,8.0,158,5.063291
5,"Home Ownership Transfer, Tenancy Management an...",9.0,173,5.202312
9,Revenue,9.0,173,5.202312
0,City Health,9.0,172,5.232558
13,Technical Services,9.0,166,5.421687
7,Property Management,5.0,92,5.434783


In [9]:
# We are looking for days which had really high service requests. 
query = '''
select *
from res
where anomaly = -1 and req > 2
'''

anom_df = sqldf.run(query)

In [10]:
query = '''
select df.*
from df
inner join anom_df on anom_df.department = df.department and anom_df.date = df.date
'''

anom_line = sqldf.run(query)

In [11]:
# Let's see why these days were picked up as anomalies
query = '''
select department,
       cause_code,
       count(distinct notification_number) cnt
from anom_line
group by department, cause_code
'''

sum_table = sqldf.run(query)
sum_table['cause_code'] = sum_table['cause_code'].fillna('unknown')

In [12]:
fig = px.bar(sum_table, x="department", y="cnt", color="cause_code", title="Causes for service: anomalous",
                 width=800, height=800)
fig.show()

# Interpretations
The top departments with the most anomalous services requests are in general the ones with the highest requests overall.
* Electricity
* Distribution
* Commercial
* Solid waste

It would appear asif Distribution has to contend with some arson which may have caused a spike in the service requests.

However, the majority of anomalous service requests have unknown (not assigned) causes.
It's hard to make recommendations if the reasons for the request is unclear.

cause_code_group could give insights as to what the service type was.

In [13]:
# investigate cause_group
query = '''
select department,
       cause_code_group,
       count(distinct notification_number) cnt
from anom_line
group by department, cause_code_group
'''
cause_group = sqldf.run(query)

cause_group['cause_code_group'] = cause_group['cause_code_group'].fillna('unknown')

fig = px.bar(cause_group, x="department", y="cnt", color="cause_code_group", title="Cause group for service: anomalous",
                 width=800, height=800)
fig.show()

This also falls in the unknown grouping mostly